In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchsummary import summary

import numpy as np
import PIL
import os
import matplotlib.pyplot as plt
import shutil



In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataDir = './dataset/AI-DATASET/data'

trainData_dir = os.path.join(dataDir, 'trainData')
train_dir = os.path.join(dataDir, 'train')
test_dir = os.path.join(dataDir, 'test')

beverage_dir = os.path.join(trainData_dir, 'beverage')
dailynecessity_dir = os.path.join(trainData_dir, 'dailynecessity')
iceCream_dir = os.path.join(trainData_dir, 'iceCream')
instanceFood_dir = os.path.join(trainData_dir, 'instanceFood')
Snack_dir = os.path.join(trainData_dir, 'Snack')


beverage_files = [f'beverage.{i}.jpg' for i in range(1,  501)]
dailynecessity_files = [f'dailynecessity.{i}.jpg' for i in range(1, 501
)]
iceCream_files = [f'iceCream.{i}.jpg' for i in range(1, 501)]
instanceFood_files = [f'instanceFood.{i}.jpg' for i in range(1, 501)]
Snack_files = [f'Snack.{i}.jpg' for i in range(1, 501)]


"""
train, valid, test 폴더 생성
"""

train_set_dir = os.path.join(train_dir, 'train')
os.mkdir(train_set_dir)
valid_set_dir = os.path.join(train_dir, 'valid')
os.mkdir(valid_set_dir)
test_set_dir = os.path.join(train_dir, 'test')
os.mkdir(test_set_dir)


for beverage, dailynecessity, iceCream, instanceFood, snack in zip(beverage_files[:300], dailynecessity_files[:300], iceCream_files[:300], instanceFood_files[:300], Snack_files[:300]):
    src = os.path.join(train_dir, beverage)
    dst = os.path.join(train_set_dir, beverage)
    shutil.move(src, dst)

    src = os.path.join(train_dir, dailynecessity)
    dst = os.path.join(train_set_dir, dailynecessity)
    shutil.move(src, dst)

    src = os.path.join(train_dir, iceCream)
    dst = os.path.join(train_set_dir, iceCream)
    shutil.move(src, dst)
    
    
    src = os.path.join(train_dir, instanceFood)
    dst = os.path.join(train_set_dir, instanceFood)
    shutil.move(src, dst)

    src = os.path.join(train_dir, snack)
    dst = os.path.join(train_set_dir, snack)
    shutil.move(src, dst)

for beverage, dailynecessity, iceCream, instanceFood, snack in zip(beverage_files[300:400], dailynecessity_files[300:400], iceCream_files[300:400], instanceFood_files[300:400], Snack_files[300:400]):
    src = os.path.join(train_dir, beverage)
    dst = os.path.join(valid_set_dir, beverage)
    shutil.move(src, dst)

    src = os.path.join(train_dir, dailynecessity)
    dst = os.path.join(valid_set_dir, dailynecessity)
    shutil.move(src, dst)

    src = os.path.join(train_dir, iceCream)
    dst = os.path.join(valid_set_dir, iceCream)
    shutil.move(src, dst)
    
    
    src = os.path.join(train_dir, instanceFood)
    dst = os.path.join(valid_set_dir, instanceFood)
    shutil.move(src, dst)

    src = os.path.join(train_dir, snack)
    dst = os.path.join(valid_set_dir, snack)
    shutil.move(src, dst)

for beverage, dailynecessity, iceCream, instanceFood, snack in zip(beverage_files[400:500], dailynecessity_files[400:500], iceCream_files[400:500], instanceFood_files[400:500], Snack_files[400:500]):
    src = os.path.join(train_dir, beverage)
    dst = os.path.join(test_set_dir, beverage)
    shutil.move(src, dst)

    src = os.path.join(train_dir, dailynecessity)
    dst = os.path.join(test_set_dir, dailynecessity)
    shutil.move(src, dst)

    src = os.path.join(train_dir, iceCream)
    dst = os.path.join(test_set_dir, iceCream)
    shutil.move(src, dst)
    
    
    src = os.path.join(train_dir, instanceFood)
    dst = os.path.join(test_set_dir, instanceFood)
    shutil.move(src, dst)

    src = os.path.join(train_dir, snack)
    dst = os.path.join(test_set_dir, snack)
    shutil.move(src, dst)
        

In [31]:

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, files, root, mode='train', transform=None):
        self.files = files
        self.root = root
        self.mode = mode
        self.transform=transform
        
        if 'beverage' in files[0]:
            self.label = 0
        elif 'dailynecessity' in files[0]:
            self.label = 1
        elif 'iceCream' in files[0]:
            self.label = 2
        elif 'instanceFood' in files[0]:
            self.label = 3
        elif 'Snack' in files[0]:
            self.label = 4
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, index):
        img = PIL.Image.open(os.path.join(self.root, self.files[index]))
        
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            return img, np.array([self.label])
        else:
            return img, self.files[index]

In [32]:
model = torchvision.models.resnet50(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 1024),
    nn.Dropout(0.2),
    nn.Linear(1024, 512),
    nn.Dropout(0.1),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model.cuda()
summary(model, input_size=(3,224,224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

모델 설정 및 정상적인 파싱 확인

In [36]:
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256,256)),
    torchvision.transforms.RandomCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,244)),
    torchvision.transforms.ToTensor(),
])
train_beverage_dataset = CustomDataset(beverage_files[:300], train_set_dir, transform=train_transform)
train_dailynecessity_dataset = CustomDataset(dailynecessity_files[:300], train_set_dir, transform=train_transform)
train_iceCream_dataset = CustomDataset(iceCream_files[:300], train_set_dir, transform=train_transform)
train_instanceFoods_dataset = CustomDataset(instanceFood_files[:300], train_set_dir, transform=train_transform)
train_Snack_dataset = CustomDataset(Snack_files[:300], train_set_dir, transform=train_transform)

train_dataset = torch.utils.data.ConcatDataset([train_beverage_dataset, train_dailynecessity_dataset, train_iceCream_dataset, train_instanceFoods_dataset, train_Snack_dataset])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

samples, labels = iter(train_loader).__next__()
classes = {0:'beverage', 1:'dailynecessity', 2:'Foods', 3:'iceCream', 4: 'instanceFood', 5: 'Snack'}
fig = plt.figure(figsize=(16,24))
for i in range(24):
    a = fig.add_subplot(4,6,i+1)
    a.set_title(classes[labels[i].item()])
    a.axis('off')
    a.imshow(np.transpose(samples[i].numpy(), (1,2,0)))
plt.subplots_adjust(bottom=0.2, top=0.6, hspace=0)


FileNotFoundError: [Errno 2] No such file or directory: './dataset/AI-DATASET/data\\train\\dailynecessity.196.jpg'